In [1]:
!pip install ipywidgets

In [2]:
!pip install mp-api

In [3]:
!pip install mpcontribs-client

In [39]:
from mp_api.client import MPRester

In [40]:
import pandas as pd
import requests

In [64]:
df = pd.read_csv('6_result.csv')
df

,Name,normalised_name,raw_value,mol_file
0,As20S60Ge20,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,NaN
1,Propionaldehyde,CCC=O,1.900,<rdkit.Chem.rdchem.Mol object at 0x00000159524...
2,PEO,OO,1.470,<rdkit.Chem.rdchem.Mol object at 0x00000159524...
3,Alumina,O=[Al]O[Al]=O,1.700,<rdkit.Chem.rdchem.Mol object at 0x00000159524...
4,cyclotriphosphazene,N1PNP=NP1,1.900,<rdkit.Chem.rdchem.Mol object at 0x00000159524...
...,...,...,...,...
1329,SiON,"[['N', 1.0], ['O', 1.0], ['Si', 1.0]]",2.010,NaN
1330,boehmite,[OH-].[O--].[Al+3],1.650,<rdkit.Chem.rdchem.Mol object at 0x00000159538...
1331,Nd2Zr2O7,"[['Nd', 2.0], ['O', 7.0], ['Zr', 2.0]]",2.110,NaN
1332,Zn(PO3)2,CC(=O)OCC1=C2C(=O)O[Zn]OC(=O)C(N)CCCC(=O)N[C@@...,1.526,<rdkit.Chem.rdchem.Mol object at 0x0000015953C...


In [42]:
MP_API_KEY = "d2mK9UPMEmGUfNpiFEf8gSwE6De4XX2i"

In [43]:
with MPRester(MP_API_KEY) as mpr:
    print(len(mpr.summary.available_fields))
    print(mpr.summary.available_fields)
    list_fields = mpr.summary.available_fields

73
['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_to

In [51]:
# пробуем на одном примере
data = mpr.get_entries('K2O')[0]
material_id = data.data["material_id"]
materials = mpr.summary.search(material_ids=[material_id])
material = materials[0]

Retrieving ThermoDoc documents:   0%|          | 0/10 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
for field in list_fields:
    print(field, 'дескриптор')
    print(getattr(material, field))

builder_meta дескриптор
emmet_version='0.38.6' pymatgen_version='2022.10.22' pull_request=825 database_version='2022.10.28' build_date=datetime.datetime(2022, 10, 28, 0, 55, 23, 76000)
nsites дескриптор
6
elements дескриптор
[Element K, Element O]
nelements дескриптор
2
composition дескриптор
K4 O2
composition_reduced дескриптор
K2 O1
formula_pretty дескриптор
K2O
formula_anonymous дескриптор
AB2
chemsys дескриптор
K-O
volume дескриптор
175.829935846865
density дескриптор
1.779175282799177
density_atomic дескриптор
29.304989307810832
symmetry дескриптор
crystal_system=<CrystalSystem.tet: 'Tetragonal'> symbol='I4_1/amd' number=141 point_group='4/mmm' symprec=0.1 version='1.16.2'
property_name дескриптор
summary
material_id дескриптор
mp-936219
deprecated дескриптор
False
deprecation_reasons дескриптор
None
last_updated дескриптор
2022-10-28 00:55:23.076000
origins дескриптор
[]
warnings дескриптор
[]
structure дескриптор
Full Formula (K4 O2)
Reduced Formula: K2O
abc   :   7.291465  10.4

In [58]:
# отбираем дескрипторы
good_list_fields = ['nsites', 'volume', 'density', 'density_atomic', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt', 'g_reuss', 'g_vrh', 'universal_anisotropy', 'homogeneous_poisson', 'e_total', 'e_ionic', 'e_electronic', 'n', 'e_ij_max', 'weighted_surface_energy_EV_PER_ANG2', 'weighted_surface_energy', 'weighted_work_function', 'surface_anisotropy', 'shape_factor', 'has_reconstructed', 'theoretical']
print(len(good_list_fields))

47


In [65]:
# создание новых колонок в датафрейме
for column_name in good_list_fields:
    df[column_name] = None

In [66]:
w8_list_MP = []
for i in range(len(df['Name'].tolist())):
    try:
        data = mpr.get_entries(df['Name'].tolist()[i])[0]
        material_id = data.data["material_id"]
        materials = mpr.summary.search(material_ids=[material_id])
        material = materials[0]
        for field in good_list_fields:
            w8_list_MP.append(getattr(material, field))
        #       перебор названий колонок и соответствующих значений
        for col, val in zip(good_list_fields, w8_list_MP):
#           добавление значения в определенную строку и колонку
            df.loc[i, col] = val
    except:
        pass

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/4684 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


Retrieving ThermoDoc documents:   0%|          | 0/11136 [00:00<?, ?it/s]

...

Retrieving ThermoDoc documents: 0it [00:00, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
df

,Name,normalised_name,raw_value,mol_file,nsites,volume,density,density_atomic,uncorrected_energy_per_atom,energy_per_atom,...,e_electronic,n,e_ij_max,weighted_surface_energy_EV_PER_ANG2,weighted_surface_energy,weighted_work_function,surface_anisotropy,shape_factor,has_reconstructed,theoretical
0,As20S60Ge20,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Propionaldehyde,CCC=O,1.900,<rdkit.Chem.rdchem.Mol object at 0x00000159524...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,PEO,OO,1.470,<rdkit.Chem.rdchem.Mol object at 0x00000159524...,6,167.149921,5.641219,27.85832,-4.066061,-4.066061,...,None,None,None,None,None,None,None,None,None,False
3,Alumina,O=[Al]O[Al]=O,1.700,<rdkit.Chem.rdchem.Mol object at 0x00000159524...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,cyclotriphosphazene,N1PNP=NP1,1.900,<rdkit.Chem.rdchem.Mol object at 0x00000159524...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,SiON,"[['N', 1.0], ['O', 1.0], ['Si', 1.0]]",2.010,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1330,boehmite,[OH-].[O--].[Al+3],1.650,<rdkit.Chem.rdchem.Mol object at 0x00000159538...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1331,Nd2Zr2O7,"[['Nd', 2.0], ['O', 7.0], ['Zr', 2.0]]",2.110,NaN,6,167.149921,5.641219,27.85832,-4.066061,-4.066061,...,None,None,None,None,None,None,None,None,None,False
1332,Zn(PO3)2,CC(=O)OCC1=C2C(=O)O[Zn]OC(=O)C(N)CCCC(=O)N[C@@...,1.526,<rdkit.Chem.rdchem.Mol object at 0x0000015953C...,6,167.149921,5.641219,27.85832,-4.066061,-4.066061,...,None,None,None,None,None,None,None,None,None,False


In [68]:
df.to_csv('db_descriptors_MaterialsProject.csv', index=False)